# Spectrogram_vgg16

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
import pickle
from keras import layers, metrics, applications
from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from keras import layers
import glob
import os
from PIL import Image
from random import shuffle
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import h5py
import tensorflow as tf
# from keras import backend as K

In [2]:
tf.__version__

'2.8.0'

In [3]:
batch_size = 64
epochs = 150

In [4]:
infile = open('./csDataframe.pkl', 'rb')
data = pickle.load(infile)
infile.close()
set = []
for row in data.itertuples():
    #x=loaded[row.File_Name]['x'][int(row.Start*loaded[row.File_Name]['sr']):int(row.End*loaded[row.File_Name]['sr'])]
    x, sr = row.npArr , 4000
    
    X = np.abs(librosa.stft(x,n_fft = 511, hop_length = 128))
    Xdb = librosa.amplitude_to_db(abs(X))

    np.reshape(Xdb,(1,256,94,1))

    
    if(row[5] == 0 and row[6] == 0): set.append((Xdb,0))
    else: set.append((Xdb,1))

x,y = zip(*set)

from sklearn.model_selection import train_test_split
combine_X, test_X, combine_y, test_y = train_test_split(
    x,
    y,
    test_size = 0.2, 
    random_state = 2018)
train_X, valid_X, train_y, valid_y = train_test_split(
    combine_X,
    combine_y,
    test_size = 0.25, 
    random_state = 2018)
print('Training', np.shape(train_X), np.shape(train_y))
print('Validation', np.shape(valid_X), np.shape(valid_y))
print('Test', np.shape(test_X), np.shape(test_y))

    

Training (4138, 256, 94) (4138,)
Validation (1380, 256, 94) (1380,)
Test (1380, 256, 94) (1380,)


In [5]:
train_X=np.array(train_X)
train_y = np.array(train_y)
valid_X= np.array(valid_X)
valid_y= np.array(valid_y)
test_X=np.array(test_X)
test_y = np.array(test_y)



In [11]:
#np.shape(train_X[1:])

In [ ]:
#train_X = np.expand_dims(train_X, axis=3)
#train_X.shape

: 

In [6]:
import tensorflow as tf
from keras.models import *
from keras.layers import *
# this is what the results are scored on, so we should keep this
def top_3_accuracy(x, y): 
    return metrics.sparse_top_k_categorical_accuracy(x,y,3)
def create_model():
    '''
    model = Sequential()
    model.add(applications.mobilenet.MobileNet(input_shape= (256,94,1),
                                    classes=1,
                                    weights=None))
  
    
    model.add(Dense(256))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(.5))

    model.add(Dense(1))
    model.add(BatchNormalization())
    model.add(Activation('sigmoid'))
    '''

    model = Sequential()
    input_shape=( 256, 94, 1)

    model.add(Conv2D(32, (7, 7), strides=(1, 1), input_shape=input_shape))
    model.add(SpatialDropout2D(.2, data_format='channels_last'))
    #model.add(BatchNormalization())
    #model.add(Dropout(rate=0.2))
    #model.add(MaxPooling2D((2, 2), strides=(1, 1)))
    #model.add(Activation('relu'))

    model.add(Conv2D(64, (5, 5), strides=(1, 1), padding="same"))
    model.add(SpatialDropout2D(.2, data_format='channels_last'))
    #model.add(BatchNormalization())
    #model.add(Dropout(rate=0.2))
    model.add(MaxPooling2D((4, 2), strides=(4, 2)))
    model.add(Activation('relu'))

    model.add(Conv2D(256, (3, 3), padding="same"))
    #model.add(BatchNormalization())
    model.add(Dropout(rate=0.2))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(Activation('relu'))

    model.add(Conv2D(256, (3, 3), padding="same"))
    #model.add(BatchNormalization())
    model.add(Dropout(rate=0.2))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(Activation('relu'))

    model.add(Conv2D(256, (3, 3), padding="same"))
    #model.add(BatchNormalization())
    model.add(Dropout(rate=0.2))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(Activation('relu'))

    model.add(Conv2D(256, (3, 3), padding="same"))
    #model.add(BatchNormalization())
    model.add(Dropout(rate=0.2))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(Activation('relu'))

    model.add(Conv2D(256, (3, 3), padding="same"))
    #model.add(BatchNormalization())
    model.add(Dropout(rate=0.2))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(Activation('relu'))

    model.add(Conv2D(512, (3, 3), padding="same"))
    #model.add(BatchNormalization())
    model.add(Dropout(rate=0.2))
    model.add(Activation('relu'))

    model.add(Flatten())
    #model.add(GlobalAveragePooling2D())

    model.add(Dense(256))
    #model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(rate=0.2))


    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.summary()
    return model


'''
model.add(Dense(64, input_dim=14, init='uniform'))
model.add(BatchNormalization())
model.add(Activation('tanh'))
model.add(Dropout(0.5))

# we can think of this chunk as the hidden layer    
model.add(Dense(64, init='uniform'))
model.add(BatchNormalization())
model.add(Activation('tanh'))
model.add(Dropout(0.5))

# we can think of this chunk as the output layer
model.add(Dense(2, init='uniform'))
model.add(BatchNormalization())
model.add(Activation('softmax'))

# setting up the optimization of our weights 
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=sgd)
'''
model = create_model()


opt = tf.optimizers.Adam(lr=1e-3)
model.compile(optimizer=opt, 
    loss=losses.binary_crossentropy, 
    metrics=['accuracy', 'Precision', 'AUC', tf.keras.metrics.Recall(), tf.keras.metrics.TrueNegatives()])




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 250, 88, 32)       1600      
                                                                 
 spatial_dropout2d (SpatialD  (None, 250, 88, 32)      0         
 ropout2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 250, 88, 64)       51264     
                                                                 
 spatial_dropout2d_1 (Spatia  (None, 250, 88, 64)      0         
 lDropout2D)                                                     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 62, 44, 64)       0         
 )                                                               
                                                        

C:\Users\Rafia.Alice\Anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [7]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
# weight_path="{}_weights.best.hdf5".format('spectro_sound_model')
checkpoint = ModelCheckpoint(filepath ="./check.hDf5", monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)
# reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', 
#                                    factor=0.8, patience=5, 
#                                    verbose=1, mode='auto', 
#                                    epsilon=0.0001, cooldown=5, 
#                                    min_lr=0.0001)
early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=15) # probably needs to be more patient, but kaggle time is limited
callbacks_list = [checkpoint, early]

In [9]:
# import tf.keras.backend as K
# K.tensorflow_backend._get_available_gpus()

In [8]:
from IPython.display import clear_output
train_X=np.array(train_X)
train_y = np.array(train_y)
fit_results = model.fit(train_X, train_y,
          epochs=epochs,
          batch_size=batch_size,
          validation_data=(valid_X,valid_y), 
          callbacks=callbacks_list)
# clear_output()

Epoch 1/150
65/65 [==============================] - ETA: 0s - loss: 0.8583 - accuracy: 0.5401 - precision: 0.5115 - auc: 0.5429 - recall: 0.4369 - true_negatives: 1387.0000
Epoch 1: val_loss improved from inf to 0.68500, saving model to .\check.hDf5
65/65 [==============================] - 401s 6s/step - loss: 0.8583 - accuracy: 0.5401 - precision: 0.5115 - auc: 0.5429 - recall: 0.4369 - true_negatives: 1387.0000 - val_loss: 0.6850 - val_accuracy: 0.5833 - val_precision: 0.6283 - val_auc: 0.6461 - val_recall: 0.3218 - val_true_negatives: 592.0000
Epoch 2/150
65/65 [==============================] - ETA: 0s - loss: 0.6846 - accuracy: 0.5776 - precision: 0.5613 - auc: 0.5892 - recall: 0.4549 - true_negatives: 1507.0000
Epoch 2: val_loss did not improve from 0.68500
65/65 [==============================] - 344s 5s/step - loss: 0.6846 - accuracy: 0.5776 - precision: 0.5613 - auc: 0.5892 - recall: 0.4549 - true_negatives: 1507.0000 - val_loss: 0.6894 - val_accuracy: 0.5203 - val_precision:

In [12]:
model.save('D:\model\\VGG.h5')
print('Saved model to disk!')
  # serialize weights to HDF5
model.save_weights("weight.h5")
print("Saved model to disk")

Saved model to disk!
Saved model to disk


In [13]:
model.load_weights("./check.hDf5")
model.evaluate(x=test_X, y=test_y)

44/44 [==============================] - 19s 436ms/step - loss: 0.5784 - accuracy: 0.7232 - precision: 0.7486 - auc: 0.7930 - recall: 0.6248 - true_negatives: 590.0000


[0.5784024596214294,
 0.7231884002685547,
 0.7486238479614258,
 0.792981743812561,
 0.6248085498809814,
 590.0]

In [10]:
# full_test_ds_pointer = h5py.File(os.path.join(feature_dir, 'test_spectrograms.h5'))
# for k in full_test_ds_pointer.keys():
#     print(k, full_test_ds_pointer[k].shape)
# test_files = [x.decode() for x in full_test_ds_pointer['fname'].value]
# test_preds = model.predict(full_test_ds_pointer['spectrograms'].value, 
#                            batch_size=batch_size, verbose=True)